In [1]:
import numpy as np
import random

class Tetris:

    def __init__(self, rows, cols, max_tiles, random_seed, agent):
        self.rows = rows
        self.cols = cols
        self.max_tiles = max_tiles
        self.random_seed = random_seed
        self.agent = agent
        
        # Create table for game board, entries 1 means occupied, entries -1 means free
        # Use type float32 to simplify conversion to tensors in torch
        self.board = np.empty((rows, cols), dtype=np.float32)
        self.current_tile = -1
        self.tile_x = -1
        self.tile_y = -1
        self.tile_orientation = -1

        # Tile set (at most 2 by 2)
        #   x       
        #   x      x x

        #   0 x    x 0
        #   x 0    0 x

        #   x x    x 0    0 x    x x
        #   x 0    x x    x x    0 x

        #   x x
        #   x x
        
        # Tile structure of tiles[i, j]: 
        # The first dimension denotes x value, 
        # the length denotes the number of columns taken by the tile. 
        # The second dimension consist of pairs giving the y range: 
        # first element in the pair is the first row of the tile and 
        # the second element second is the last row plus 1 of the tile 
        # for the current column
        self.tiles = [
            [
                [[0, 2]], 
                [[0, 1], [0, 1]],
            ],
            [
                [[0, 1], [1, 2]], 
                [[1, 2], [0, 1]],
            ],
            [
                [[0, 2], [1, 2]], 
                [[0, 2], [0, 1]], 
                [[0, 1], [0, 2]], 
                [[1, 2], [0, 2]],
            ],
            [
                [[0, 2], [0, 2]],
            ],
        ]

        if self.agent is not None:
            self.agent.init_board(self)

        self.start()

    def start(self):
        self.gameover = False
        self.tile_count = 0
        self.board.fill(-1)
        if self.random_seed:
            random.seed(self.random_seed)
        self.new_tile()
        
    def new_tile(self):
        if self.tile_count < self.max_tiles:
            self.current_tile = random.randint(0, len(self.tiles) - 1)
            self.tile_count += 1
        else:
            self.gameover = True
        self.tile_x = self.cols // 2
        self.tile_y = self.rows
        self.tile_orientation = 0
        '''self.agent.get_state()'''

    def move(self, new_tile_x, new_tile_orientation):
        if new_tile_orientation >= len(self.tiles[self.current_tile]):
            return 1
        old_tile_x = self.tile_x
        old_tile_orientation = self.tile_orientation
        self.tile_x = new_tile_x
        self.tile_orientation = new_tile_orientation
        if self.check_boundary():
            self.tile_x = old_tile_x
            self.tile_orientation = old_tile_orientation
            return 1
        return 0

    def check_boundary(self):
        for xLoop in range(len(self.tiles[self.current_tile][self.tile_orientation])):
            curx = self.tile_x + xLoop
            if (curx < 0) or (curx > self.cols-1):
                return 1
        return 0

    def drop(self):
        curtile = self.tiles[self.current_tile][self.tile_orientation]
        
        # Find first location where the piece collides with occupied locations on the game board
        self.tile_y = 0
        for xLoop in range(len(curtile)):
            curx = (self.tile_x + xLoop) % self.cols
            # Find first occupied location in this column            
            cury = -1
            for yLoop in range(self.rows -1, -1, -1):
                if self.board[yLoop, curx] > 0:
                    # Calculate the y position for this column if no other columns are taken into account
                    cury = yLoop + 1 - curtile[xLoop][0]
                    break
            # Use the largest y position for all columns of the tile
            if self.tile_y < cury:
                self.tile_y = cury

        # Change board entries at the newly placed tile to occupied
        for xLoop in range(len(curtile)):
            if self.tile_y + curtile[xLoop][1] > self.rows:
                self.gameover = 1
                return -100;
            else:
                self.board[self.tile_y + curtile[xLoop][0]:self.tile_y + curtile[xLoop][1], (xLoop + self.tile_x) % self.cols] = 1

        # Remove full lines
        lineCount = 0
        for yLoop in range(self.rows - 1, -1, -1):
            if np.sum(np.array(self.board[yLoop,:]) > 0) == self.cols:
                lineCount += 1
                for y1Loop in range(yLoop,self.rows - 1):
                    self.board[y1Loop, :] = self.board[y1Loop + 1, :]
                self.board[self.rows - 1, :] = -1
        if lineCount > 0:
            curReward = 10 ** (lineCount - 1)
        else:
            curReward = 0
        # Choose the next tile
        self.new_tile()

        return curReward

In [2]:
import pygame

class HumanPlayer:

    def init_board(self, tetris):
        '''self.episode = 0'''
        self.rewards = 0
        self.tetris = tetris

    '''def get_state(self):
        pass'''

    def next_turn(self, pygame):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit("Game terminated")
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.rewards = 0
                    self.tetris.start()
                if not self.tetris.gameover:
                    if event.key == pygame.K_LEFT:
                        self.tetris.move(
                            self.tetris.tile_x - 1,
                            self.tetris.tile_orientation
                        )
                    elif event.key == pygame.K_RIGHT:
                        self.tetris.move(
                            self.tetris.tile_x + 1,
                            self.tetris.tile_orientation
                        )
                    elif event.key == pygame.K_UP:
                        self.tetris.move(
                            self.tetris.tile_x, 
                            ((self.tetris.tile_orientation + 1) 
                             % len(self.tetris.tiles[self.tetris.current_tile]))
                        )
                    elif (event.key == pygame.K_DOWN):
                        self.rewards += self.tetris.drop()


# to play the game, use arrow keys
you = HumanPlayer()

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
tetris = Tetris(rows=4, cols=4, max_tiles=50, random_seed=123456, agent=you)

In [4]:
# Define some colors for painting
BLACK = (0, 0, 0)
GREY = (128, 128, 128)
WHITE = (255, 255, 255)
RED =  (255, 0, 0)

TILE_SIZE = 20

# Initialize the game engine
pygame.init()
screen = pygame.display.set_mode((200 + tetris.cols * TILE_SIZE, 200 + tetris.rows * TILE_SIZE))
'''clock = pygame.time.Clock()'''
pygame.key.set_repeat(300, 100)
pygame.display.set_caption('Turn-based tetris')
font = pygame.font.SysFont("Calibri", 25, True)
'''framerate = 0;'''

# Loop until the window is closed
while True:
    '''if isinstance(tetris.agent, HumanPlayer):'''
    tetris.agent.next_turn(pygame)
    '''else:
        pygame.event.pump()
        for event in pygame.event.get():
            if event.type==pygame.KEYDOWN:
                if event.key==pygame.K_SPACE:
                    if framerate > 0:
                        framerate=0
                    else:
                        framerate=10
                if (event.key == pygame.K_LEFT) and (framerate > 1):
                    framerate -= 1
                if event.key == pygame.K_RIGHT:
                    framerate += 1
        tetris.agent.next_turn()'''

    if pygame.display.get_active():
        # Paint game board
        screen.fill(WHITE)

        for i in range(tetris.rows):
            for j in range(tetris.cols):
                pygame.draw.rect(
                    screen, 
                    GREY, 
                    [100 + TILE_SIZE * j, 80 + TILE_SIZE * (tetris.rows- i ), TILE_SIZE, TILE_SIZE], 
                    1
                )
                if tetris.board[i][j] > 0:
                    pygame.draw.rect(
                        screen, 
                        BLACK,
                        [101 + TILE_SIZE * j, 81 + TILE_SIZE * (tetris.rows - i), TILE_SIZE - 2, TILE_SIZE - 2]
                    )

        if tetris.current_tile is not None:
            curTile = tetris.tiles[tetris.current_tile][tetris.tile_orientation]
            for xLoop in range(len(curTile)):
                for yLoop in range(curTile[xLoop][0], curTile[xLoop][1]):
                    pygame.draw.rect(
                        screen,
                        RED,
                        [101 + TILE_SIZE * ((xLoop + tetris.tile_x) % tetris.cols), 
                         81 + TILE_SIZE * (tetris.rows - (yLoop + tetris.tile_y)), 
                         TILE_SIZE - 2, 
                         TILE_SIZE - 2]
                    )

        screen.blit(font.render(f"Reward: {you.rewards}", True, BLACK), [0, 0])
        screen.blit(font.render(f"Tile {tetris.tile_count}/{tetris.max_tiles}", True, BLACK), [0, 30])
        '''if framerate > 0:
            screen.blit(font.render("FPS: "+str(framerate),True,BLACK),[320,0])
        screen.blit(font.render("Reward: "+str(you.rewards),True,BLACK),[0,0])'''
        if tetris.gameover:
            screen.blit(font.render("G A M E   O V E R", True, RED), [40, 100 + tetris.rows * TILE_SIZE])
            screen.blit(font.render("Press ESC to try again", True, RED), [0, 100 + tetris.rows * TILE_SIZE + 30])

        pygame.display.flip()
        '''clock.tick(framerate)'''

SystemExit: Game terminated

/Users/giovannivolpe/miniconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
